# Midterm

__Done by: Mandy Gu__

__Import all packages needed__

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import os
import glob
import json
import mysql.connector

In [2]:
# Save the path of current directory for further use
current_dir = '/Users/qimenggu/Desktop/UC Davis/Data Design/Midterm'

# Part 1

__a) Write a program that accesses all trivia results for the numbers from 0 (zero) to 100 using batch requests only (One single query for all the numbers). Print the output result to the screen in the format of [3-digit number with leading zeros] - [TRIVIA] (One number per each line).__

In [3]:
def num2trivia(url, RequestType, NumOfDigits, Number):
    '''Input a list of number, Parse the corresponding trivia in a specified format from the "numberapi" website'''
    pages = requests.get(url + str(Number) + RequestType)
    content = BeautifulSoup(pages.text)
    trivia = content.find('p').text.strip()
    output = str(number).zfill(NumOfDigits) + ' - ' + trivia
    return output

# Parse a list of 0-99 to the website and transfer it to a 3-digit number with leading zeros
for number in range(100):
    output = num2trivia('http://numbersapi.com/', '/trivia', 3, number)
#     print (output)

AttributeError: 'NoneType' object has no attribute 'text'

# Part 2

__a) How would you modify your search query to only include buy-it-now (non-auction) items? What is the GET request's variable name corresponding to buy-it-now searches? How would you modify your search query to include 100 items per search result page? What is the GET request's variable name corresponding to items per page searches? Include your answers as a comment in your code marking it as Part 2-a.__

In [23]:
# Variable name for buy-it-now: LH_BIN
# https://www.ebay.com/sch/i.html?_nkw=samsung+tv&LH_BIN=1
# Variable name for the number of items per search result page: _ipg
# https://www.ebay.com/sch/i.html?_nkw=samsung+tv&LH_BIN=1&_ipg=100

__b) Modify your previous code to search for buy-it-now listings of "canon eos 5d" and limit the number of items to 100 per page.__

__Identify all the sponsored items.__

__For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)__

In [24]:
sponsor_url = ''
non_sponsor_url = ''
for i in range(10):    
    url = 'https://www.ebay.com/sch/i.html?&_nkw=playstation+4+slim&LH_BIN=1&_ipg=100&_pgn=' + str(i + 1)
    headers = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    a_href = soup.select("li.s-item")
    for href in a_href: # Go through all items on this page   
        a = href.find("a")             
        if not a:                    
            continue
        
        url_simplify = re.sub("(.*)\\?.*", r"\1", a.get('href')) # Obtain a shorter url
        if href.find('span', {'role': 'text'}) != None:
            text = href.find('span', {'role': 'text'}).get_text() # 'Sponsor' is found inside this <span>
        
            if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*', text):
                sponsor_url += url_simplify + '\n' # Identified as sponsor items
            else:
                non_sponsor_url += url_simplify + '\n'
        else:
            non_sponsor_url += url_simplify + '\n' # Identified as non-sponsor items
    time.sleep(2)
    
file = open('sponsored.txt', 'wt', encoding = "utf-8")
file.write(sponsor_url)
file.close()

file = open('non-sponsored.txt', 'wt', encoding = "utf-8")
file.write(non_sponsor_url)
file.close()

__c) Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files in (b) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "(item-id)htm" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).__

In [25]:
def create_folder(FolderName):
    '''Create a new folder if not exists'''
    if not os.path.exists(FolderName):
        os.makedirs(FolderName)

# Create a folder for sponsored items
create_folder('sponsored')
# Create a folder for non-sponsored items
create_folder('non-sponsored')

In [26]:
def extract_url(txt):
    '''Extract all urls in a txt file, which is seperated by \n'''
    file = open(txt, 'r')
    content = file.read().strip().split('\n')
    file.close()
    return content

def parse_url(url):
    '''Assign the itemid as its filename and parse the webpage by url'''
    r = re.compile(r'/([0-9]{12})')
    filename = str(r.findall(url)[0]) + '.htm'
    # In case some errors occur when parsing
    try:
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.text)
    except:
        print ('Fail to parse: ',url)
    return soup, filename

def url2file(folder, filename, soup):
    '''Open a new file under specific folder named by itemid and write its corresponding webpage content into it'''
    file = open(os.path.join(folder, filename), 'wt')
    file.write(str(soup))
    file.close()
    
# Find all txt in the currect directory
for txt in glob.glob('*.txt'):
    content = extract_url(txt) # Extract all urls
    for url in content:
        soup = parse_url(url)[0] # Parse webpage content from the url
        filename = parse_url(url)[1] # Get the filename
        folder = txt.split('.')[0] # Determin if it is sponsored or non-sponsored
        url2file(folder, filename, soup) # Write content into corresponding file under specific folder
        time.sleep(2) # Act as a human in case of codes going off

__d) Write a separate piece of code that loops through the pages you downloaded in (c) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:__

__seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).__

__In your code, highlight the selector command you choose to obtain each element using comments.__

In [27]:
def sellerName(RightSummaryPanel):
    '''Find content on RightSummaryPanel containing a unique id, search for seller name by regex and clean the data'''
    if re.findall(r'Member ID:(.*?)"', str(RightSummaryPanel)) != []:
        seller_name = re.findall(r'Member ID:(.*?)"', str(RightSummaryPanel))[0].replace('\xa0', '')
    else:
        seller_name = None
    return seller_name

def sellerScore(RightSummaryPanel):
    '''Find content on RightSummaryPanel, search for seller score by regex and make it an integer'''
    if re.findall(r'"feedback score: ([0-9]+)"', str(RightSummaryPanel)) != []:
        sellerScore = int(re.findall(r'"feedback score: ([0-9]+)"', str(RightSummaryPanel))[0])
    else:
        sellerScore = None
    return sellerScore

def itemPrice(soup):
    '''Find item price by its unique id "prcIsum", and unite all kinds of currency to US cents as an integer'''
    if soup.find('span', {'id': 'prcIsum'}) != None:
        itemPrice = soup.find('span', {'id': 'prcIsum'}).get_text()
        if re.findall(r'US.*', itemPrice):
            itemPrice = int(float(re.findall(r'[.0-9]+', itemPrice)[0]) * 100)
        elif re.findall(r'C.*', itemPrice):
            # The exchange rate is obtained online on Feb 6th
            itemPrice = int(float(re.findall(r'[.0-9]+', itemPrice)[0]) * 0.75 * 100)
        elif re.findall(r'GBP.*', itemPrice):
            itemPrice = int(float(re.findall(r'[.0-9]+', itemPrice)[0]) * 1.29 * 100)
        else:
            itemPrice = itemPrice
    else:
        itemPrice = None
    return itemPrice

def title(soup):
    '''Find title of the item by its unique id "itemTitle", and do some cleaning'''
    if soup.find('h1', {'id': 'itemTitle'}) != None:
        title = soup.find('h1', {'id': 'itemTitle'}).get_text().split('\xa0')[1]
    else:
        title = None
    return title

def itemSold(soup):
    '''Find the number of item sold on specific panel of "soldwithfeedback". If not exists, then assign None'''
    if soup.find('span', {'class': 'soldwithfeedback'}) != None:
        item_sold = soup.find('span', {'class': 'soldwithfeedback'}).get_text()
        itemSold = int(re.findall(r'([0-9]+) sold', str(item_sold))[0])
    else:
        itemSold = None
    return itemSold

def bestOffer(soup):
    '''Determin if an item can be made an offer. If its unique id does not exist, then assign 0, otherwise assign 1'''
    if soup.find('a', {'id': 'boBtn_btn'}) != None:
        bestOffer = 1
    else:
        bestOffer = 0
    return bestOffer

def returnAllowed(soup):
    '''Find the policy about return by its unique id and assign names to different kinds of return policy'''
    if soup.find('span', {'id': 'vi-ret-accrd-txt'}) != None:
        returnAllowed = soup.find('span', {'id': 'vi-ret-accrd-txt'}).get_text().replace('\xa0', ' ')
        if re.findall(r'14.*shipping', returnAllowed) != []:
            returnAllowed = '14 Buyer pays' # 14 day returns. Buyer pays for return shipping
        elif re.findall(r'30.*shipping', returnAllowed) != []:
            returnAllowed = '30 Buyer pays' # 30 day returns. Buyer pays for return shipping
        elif re.findall(r'60.*shipping', returnAllowed) != []:
            returnAllowed = '60 Buyer pays' # 60 day returns. Buyer pays for return shipping
        elif re.findall(r'Free 30.*', returnAllowed) != []:
            returnAllowed = 'Free 30' # Free 30 day returns
        elif re.findall(r'Free 60.*', returnAllowed) != []:
            returnAllowed = 'Free 60' # Free 60 day returns
        else:
            returnAllowed = 'ReturnNotAllowed' # Seller does not accept returns
    else:
        returnAllowed = None
    return returnAllowed

def shippingPrice(soup):
    '''Find shipping price in its unique id "shippingSection", unite currency and determine if it needs calculation'''
    if soup.find('div', {'id': 'shippingSection'}) != None:
        try:
            # The shipping price is in a table
            shipping_price = soup.find('div', {'id': 'shippingSection'}).find('td').get_text().strip()
            if re.findall(r'US.*', shipping_price):
                shippingPrice = int(float(re.findall(r'[.0-9]+', shipping_price)[0]) * 100)
            elif re.findall(r'C.*', shipping_price):
                shippingPrice = int(float(re.findall(r'[.0-9]+', shipping_price)[0]) * 0.75 * 100)
            elif re.findall(r'GBP.*', shipping_price):
                shippingPrice = int(float(re.findall(r'[.0-9]+', shipping_price)[0]) * 1.29 * 100)
            # You can contact your seller to arrange to pay for and pick up your item OR free shipping
            elif re.findall(r'Free.*', shipping_price) or re.findall(r'contact your seller', shipping_price):
                shippingPrice = 0
            else:
                shippingPrice = shipping_price
            # If the shipping price requires zip code to be calculated, it will not have this id
            calculate_needed = 0
        except:
            calculate_needed = 1
            shippingPrice = None
    else:
        shippingPrice = None
        calculate_needed = 0
    return shippingPrice, calculate_needed

def condition(soup):
    '''Condition may be selected in two areas, where both have their unique ids. Do some cleaning and assign names'''
    if soup.find('span', {'id': 'vi-cond-addl-info'}) != None:
        condition = soup.find('span', {'id': 'vi-cond-addl-info'}).get_text().strip()
        condition_more = soup.find('span', {'id': 'hiddenContent'}).get_text().strip()
        total_condition = condition + condition_more
    else:
        condition = soup.find('div', {'id': 'itmCondDscOly'})
        if condition != None:
            condition = condition.get_text()
            total_condition = condition.strip().split('\n')[0]
        else:
            total_condition = None
       
    if total_condition != None:
        if re.findall(r'restored', total_condition) != []:
            total_condition = 'Restored' # Restored but in excellent condition
        elif re.findall(r'used previously', total_condition) != []:
            total_condition = 'Used' # Used Previously
        elif re.findall(r'excellent, new condition', total_condition) != []:
            total_condition = 'Factory new' # Excellent new condition maybe a factory second
        else:
            total_condition = 'Brand new' # Brand new with original packaging
    return total_condition

In [28]:
def selector(dictionary):
    '''Read the webpage content and parse all elements selected to a dictionary for further use'''
    for htm in glob.glob('*.htm'):
        dictionary[htm] = {}
        file = open(htm, 'r', encoding = 'utf-8')
        soup = BeautifulSoup(file.read())
        file.close()
        
        RightSummaryPanel = soup.find('div', {'id': 'RightSummaryPanel'}) # Used for select seller name and score
        dictionary[htm]['sellerName'] = sellerName(RightSummaryPanel)
        dictionary[htm]['sellerScore'] = sellerScore(RightSummaryPanel)
        dictionary[htm]['itemPrice'] = itemPrice(soup)
        dictionary[htm]['title'] = title(soup)
        dictionary[htm]['itemSold'] = itemSold(soup)
        dictionary[htm]['bestOffer'] = bestOffer(soup)
        dictionary[htm]['returnAllowed'] = returnAllowed(soup)
        dictionary[htm]['shippingPrice'] = shippingPrice(soup)[0]
        dictionary[htm]['calculate_needed'] = shippingPrice(soup)[1]
        dictionary[htm]['condition'] = condition(soup)
    return dictionary

In [29]:
sponsor_info = {}
non_sponsor_info = {}

# Make sure it is in current directory but not in either folders
os.chdir(current_dir)
# Get into non-sponsored folder
os.chdir(os.getcwd() + '/non-sponsored')
# Put all information to dictionary
non_sponsor_info = selector(non_sponsor_info)

os.chdir(current_dir)
os.chdir(os.getcwd() + '/sponsored')
sponsor_info = selector(sponsor_info)

In [30]:
os.chdir(current_dir)

def dict2json(dictionary, file):
    '''Store dictionary to a json file for further use'''
    with open(file, 'wt') as fp:
        json.dump(dictionary, fp)

dict2json(sponsor_info, 'sponsor.json')
dict2json(non_sponsor_info, 'non_sponsor.json')

__e) Use your code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Create a database and name it "eBay". Save the information of items in (d)  into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.  If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert $12.34 into 1234 and $12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.__

In [31]:
# Load json file
with open('sponsor.json') as f:
    sponsor_info = json.load(f)

with open('non_sponsor.json') as f:
    non_sponsor_info = json.load(f)

Test for the length of all strings to assign an optimal data type

In [32]:
# max([len(sponsor_info[url]['sellerName']) for url in list(sponsor_info.keys())])

In [33]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True)
mycursor = mydb.cursor()

mycursor.execute("""DROP DATABASE IF EXISTS eBay""")
mycursor.execute("""CREATE DATABASE eBay""")
mycursor.execute("""USE eBay""")

mycursor.execute("DROP TABLE IF EXISTS eBay_items")
mycursor.execute("""CREATE TABLE eBay_items
(item_id VARCHAR(20),
sellerName VARCHAR(40),
sellerScore INT,
itemPrice INT,
title VARCHAR(80),
itemSold INT,
bestOffer INT,
returnAllowed VARCHAR(50),
shippingPrice VARCHAR(255),
shipNeedCalculate INT,
itemCondition TEXT,
sponsorOrNot VARCHAR(20)
)""")
mydb.commit()

into = "INSERT INTO eBay_items(item_id, sellerName, sellerScore, itemPrice, title, itemSold, bestOffer, returnAllowed, shippingPrice, shipNeedCalculate, itemCondition, sponsorOrNot) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

# All names of elements selected
spon_col = list(sponsor_info[list(sponsor_info.keys())[0]].keys())

# Save information of items in sponsor folder to MySQL
for item in list(sponsor_info.keys()):
    value = []
    value.append(item.replace('.htm', '')) # Insert itemid
    for col in spon_col:
        value.append(sponsor_info[item][col]) # Insert all elements selected
    value.append('Sponsor')
    mycursor.execute(into, value) # Insert if it is sponsor
    mydb.commit()

for item in list(non_sponsor_info.keys()):
    value = []
    value.append(item.replace('.htm', ''))
    for col in spon_col:
        value.append(non_sponsor_info[item][col])
    value.append('Non-Sponsor')
    mycursor.execute(into, value)
    mydb.commit()

mydb.close()

__f) Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations.__

In [35]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT sponsorOrNot, itemCondition, ROUND(AVG(sellerScore), 2) AS 'Mean Of sellerScore', ROUND(STD(sellerScore), 2) AS 'SD Of sellerScore', MIN(sellerScore) AS 'Min Of sellerScore', MAX(sellerScore) AS 'Max Of sellerScore'
FROM eBay_items
GROUP BY sponsorOrNot, itemCondition
HAVING AVG(sellerScore) IS NOT NULL AND itemCondition IS NOT NULL
ORDER BY sponsorOrNot, itemCondition
""")

results = mycursor.fetchall()

# Print the output in a pretty table
widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]))
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+--------------+---------------+---------------------+-------------------+--------------------+--------------------+
| sponsorOrNot | itemCondition | Mean Of sellerScore | SD Of sellerScore | Min Of sellerScore | Max Of sellerScore |
+--------------+---------------+---------------------+-------------------+--------------------+--------------------+
| Non-Sponsor  | Brand new     | 16093.03            | 67585.12          | 0                  | 916850             |
| Non-Sponsor  | Factory new   | 23146.90            | 109321.34         | 0                  | 638522             |
| Non-Sponsor  | Restored      | 340916.61           | 776754.82         | 0                  | 2265479            |
| Non-Sponsor  | Used          | 53432.23            | 75715.11          | 0                  | 224441             |
| Sponsor      | Brand new     | 13959.04            | 81872.13          | 0                  | 568381             |
| Sponsor      | Factory new   | 13201.00            | 25647.2  

In [36]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT sponsorOrNot, itemCondition, ROUND(AVG(itemPrice), 2) AS 'Mean of itemPrice', ROUND(STD(itemPrice), 2) AS 'SD Of itemPrice', MIN(itemPrice) AS 'Min of itemPrice', MAX(itemPrice) AS 'Max of itemPrice'
FROM eBay_items
GROUP BY sponsorOrNot, itemCondition
HAVING AVG(itemPrice) IS NOT NULL AND itemCondition IS NOT NULL
ORDER BY sponsorOrNot, itemCondition
""")

results = mycursor.fetchall()

widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]))
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+--------------+---------------+-------------------+-----------------+------------------+------------------+
| sponsorOrNot | itemCondition | Mean of itemPrice | SD Of itemPrice | Min of itemPrice | Max of itemPrice |
+--------------+---------------+-------------------+-----------------+------------------+------------------+
| Non-Sponsor  | Brand new     | 31514.58          | 8921.44         | 100              | 69999            |
| Non-Sponsor  | Factory new   | 27482.76          | 6889.45         | 19000            | 52995            |
| Non-Sponsor  | Restored      | 23355.39          | 4506.2          | 17499            | 36995            |
| Non-Sponsor  | Used          | 23110.24          | 5055.16         | 13999            | 50000            |
| Sponsor      | Brand new     | 33342.28          | 9672.23         | 22100            | 59999            |
| Sponsor      | Factory new   | 25659.00          | 4849.3          | 18995            | 33500            |
| Sponsor      | Re

In [37]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT sponsorOrNot, itemCondition, ROUND(AVG(itemSold), 2) AS 'Mean of itemSold', ROUND(STD(itemSold), 2) AS 'SD of itemSold', MIN(itemSold) AS 'Min of itemSold', MAX(itemSold) AS 'Max of itemSold'
FROM eBay_items
GROUP BY sponsorOrNot, itemCondition
HAVING AVG(itemSold) IS NOT NULL AND itemCondition IS NOT NULL
ORDER BY sponsorOrNot, itemCondition
""")

results = mycursor.fetchall()

widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]))
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+--------------+---------------+------------------+----------------+-----------------+-----------------+
| sponsorOrNot | itemCondition | Mean of itemSold | SD of itemSold | Min of itemSold | Max of itemSold |
+--------------+---------------+------------------+----------------+-----------------+-----------------+
| Non-Sponsor  | Brand new     | 155.16           | 243.07         | 11              | 896             |
| Non-Sponsor  | Factory new   | 67.00            | 52.0           | 15              | 119             |
| Non-Sponsor  | Restored      | 90.83            | 103.78         | 11              | 316             |
| Non-Sponsor  | Used          | 17.00            | 0.0            | 17              | 17              |
| Sponsor      | Brand new     | 90.14            | 95.55          | 25              | 316             |
| Sponsor      | Restored      | 739.00           | 0.0            | 739             | 739             |
| Sponsor      | Used          | 29.00            | 21.

In [38]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT sponsorOrNot, itemCondition, ROUND(AVG(bestOffer), 2) AS 'Mean of bestOffer', ROUND(STD(bestOffer), 2) AS 'SD of bestOffer', MIN(bestOffer) AS 'Min of bestOffer', MAX(bestOffer) AS 'Max of bestOffer'
FROM eBay_items
GROUP BY sponsorOrNot, itemCondition
HAVING AVG(bestOffer) IS NOT NULL AND itemCondition IS NOT NULL
ORDER BY sponsorOrNot, itemCondition
""")

results = mycursor.fetchall()

widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]))
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+--------------+---------------+-------------------+-----------------+------------------+------------------+
| sponsorOrNot | itemCondition | Mean of bestOffer | SD of bestOffer | Min of bestOffer | Max of bestOffer |
+--------------+---------------+-------------------+-----------------+------------------+------------------+
| Non-Sponsor  | Brand new     | 0.30              | 0.46            | 0                | 1                |
| Non-Sponsor  | Factory new   | 0.55              | 0.5             | 0                | 1                |
| Non-Sponsor  | Restored      | 0.33              | 0.47            | 0                | 1                |
| Non-Sponsor  | Used          | 0.40              | 0.49            | 0                | 1                |
| Sponsor      | Brand new     | 0.30              | 0.46            | 0                | 1                |
| Sponsor      | Factory new   | 0.80              | 0.4             | 0                | 1                |
| Sponsor      | Re

In [41]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT sponsorOrNot, itemCondition, ROUND(AVG(shippingPrice), 2) AS 'Mean of shipPrice', ROUND(STD(shippingPrice), 2) AS 'SD of shipPrice', MIN(shippingPrice) AS 'Min of shipPrice', MAX(shippingPrice) AS 'Max of shipPrice'
FROM eBay_items
GROUP BY sponsorOrNot, itemCondition
HAVING AVG(shippingPrice) IS NOT NULL AND itemCondition IS NOT NULL
ORDER BY sponsorOrNot, itemCondition
""")

results = mycursor.fetchall()

widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]))
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+--------------+---------------+-------------------+-----------------+------------------+------------------+
| sponsorOrNot | itemCondition | Mean of shipPrice | SD of shipPrice | Min of shipPrice | Max of shipPrice |
+--------------+---------------+-------------------+-----------------+------------------+------------------+
| Non-Sponsor  | Brand new     | 286.74            | 824.74          | 0                | 999              |
| Non-Sponsor  | Factory new   | 195.57            | 638.01          | 0                | 2500             |
| Non-Sponsor  | Restored      | 194.88            | 673.86          | 0                | 2999             |
| Non-Sponsor  | Used          | 367.25            | 795.68          | 0                | 999              |
| Sponsor      | Brand new     | 370.22            | 1135.52         | 0                | 500              |
| Sponsor      | Factory new   | 0.0               | 0.0             | 0                | 0                |
| Sponsor      | Re

In [43]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT sponsorOrNot, itemCondition, ROUND(AVG(shipNeedCalculate), 2) AS 'Mean of NeedCal', ROUND(STD(shipNeedCalculate), 2) AS 'SD of NeedCal', MIN(shipNeedCalculate) AS 'Min of NeedCal', MAX(shipNeedCalculate) AS 'Max of NeedCal'
FROM eBay_items
GROUP BY sponsorOrNot, itemCondition
HAVING itemCondition IS NOT NULL
ORDER BY sponsorOrNot, itemCondition
""")

results = mycursor.fetchall()

widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]))
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+--------------+---------------+-----------------+---------------+----------------+----------------+
| sponsorOrNot | itemCondition | Mean of NeedCal | SD of NeedCal | Min of NeedCal | Max of NeedCal |
+--------------+---------------+-----------------+---------------+----------------+----------------+
| Non-Sponsor  | Brand new     | 0.22            | 0.42          | 0              | 1              |
| Non-Sponsor  | Factory new   | 0.62            | 0.49          | 0              | 1              |
| Non-Sponsor  | Restored      | 0.33            | 0.47          | 0              | 1              |
| Non-Sponsor  | Used          | 0.29            | 0.45          | 0              | 1              |
| Sponsor      | Brand new     | 0.21            | 0.41          | 0              | 1              |
| Sponsor      | Factory new   | 0.40            | 0.49          | 0              | 1              |
| Sponsor      | Restored      | 0.33            | 0.47          | 0              | 1      

In [44]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT returnAllowed, COUNT(returnAllowed) AS 'Num of Category'
FROM eBay_items
GROUP BY returnAllowed
HAVING returnAllowed IS NOT NULL
ORDER BY COUNT(returnAllowed)
""")

results = mycursor.fetchall()

widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]) + 5)
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+--------------------+----------------------+
| returnAllowed      | Num of Category      |
+--------------------+----------------------+
| 60 Buyer pays      | 5                    |
| Free 60            | 6                    |
| 14 Buyer pays      | 38                   |
| 30 Buyer pays      | 194                  |
| Free 30            | 320                  |
| ReturnNotAllowed   | 555                  |
+--------------------+----------------------+


In [45]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT itemCondition, COUNT(itemCondition) AS 'Num of Category'
FROM eBay_items
GROUP BY itemCondition
HAVING itemCondition IS NOT NULL
ORDER BY COUNT(itemCondition)
""")

results = mycursor.fetchall()

widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]) + 5)
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+--------------------+----------------------+
| itemCondition      | Num of Category      |
+--------------------+----------------------+
| Restored           | 42                   |
| Factory new        | 65                   |
| Brand new          | 412                  |
| Used               | 601                  |
+--------------------+----------------------+


In [46]:
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='eBay')
mycursor = mydb.cursor()
mycursor.execute("""
SELECT sponsorOrNot, COUNT(sponsorOrNot) AS 'Num of Category'
FROM eBay_items
GROUP BY sponsorOrNot
HAVING sponsorOrNot IS NOT NULL
ORDER BY COUNT(sponsorOrNot)
""")

results = mycursor.fetchall()

widths = []
columns = []
tavnit = '|'
separator = '+' 

for cd in mycursor.description:
    widths.append(len(cd[0]) + 5)
    columns.append(cd[0])

for w in widths:
    tavnit += " %-"+"%ss |" % (w,)
    separator += '-'*w + '--+'

print(separator)
print(tavnit % tuple(columns))
print(separator)
for row in results:
    print(tavnit % row)
print(separator)
    
mydb.commit()
mydb.close()

+-------------------+----------------------+
| sponsorOrNot      | Num of Category      |
+-------------------+----------------------+
| Sponsor           | 123                  |
| Non-Sponsor       | 997                  |
+-------------------+----------------------+


__g) Use the stats in (f) and tell in your own words, how sponsored and non-sponsored items appear to be different. Is there a variable/column that can be used to predict the sponsor/non-sponsor items. (You do NOT need to run any model or statistic calculations for this part. Make your judgment only by looking at the stats you printed in (f))__

__Ans:__

The Score of Seller, Price of the item and the number of having a best offer of Sponsor items are somehow higher than these of non-sponsor items. 

Meanwhile, shipping prices of sponsor items, especially for restored and factory second items which are free to ship, are lower than these of non-sponsor items. But it should be noticed that shipping price of used sponsor items is more than 3 times more expensive than used non-sponsor items. Therefore, the shipping price of an used, factory second or restored item may be a determinant variable of whether the item is sponsored.